In [1]:
import scib
import anndata as ad
import pandas as pd
import numpy as np
import os
from multiprocessing import Pool 
from scipy.io import mmread
from scipy.sparse import csr_matrix
import muon
import scarches as sca
import scanpy as sc
from scib_metrics.benchmark import Benchmarker
import scib_metrics
from typing import Optional

import warnings
warnings.filterwarnings("ignore")

/root/miniconda3/envs/metric-full/lib/python3.11/site-packages/umap/__init__.py:9: ImportWarning: Tensorflow not installed; ParametricUMAP will be unavailable
  warn(
/root/miniconda3/envs/metric-full/lib/python3.11/site-packages/docrep/decorators.py:43: SyntaxWarning: 'param_categorical_covariate_keys' is not a valid key!
  doc = func(self, args[0].__doc__, *args[1:], **kwargs)
 captum (see https://github.com/pytorch/captum).
/root/miniconda3/envs/metric-full/lib/python3.11/site-packages/scanpy/_settings.py:488: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  IPython.display.set_matplotlib_formats(*ipython_format)


In [2]:
rna = ad.read_h5ad('../data/GSE126074/GSE126074-RNA_pp.h5ad')
rna.layers["counts"] = rna.X.copy()
# rna

In [3]:
atac = ad.read_h5ad('../data/GSE126074/GSE126074-ATAC_pp.h5ad')
# atac 9190 × 241757

In [4]:
atac.layers['counts'] = atac.X.copy()
sc.pp.normalize_total(atac, target_sum=1e4)
sc.pp.log1p(atac)
atac.layers['log-norm'] = atac.X.copy()
sc.pp.highly_variable_genes(atac, n_top_genes=30000)
atac = atac[:, atac.var.highly_variable].copy()

In [5]:
rna.obs_names = rna.obs_names.str.replace(r"_RNA$", "", regex=True)
atac.obs_names = atac.obs_names.str.replace(r"_ATAC$", "", regex=True)

adata = ad.concat([rna, atac], axis=1)
# adata
# AnnData object with n_obs × n_vars = 9190 × 270687
#     obs: 'cell_type', 'protocol', 'dataset', 'cluster', 'batch'
#     var: 'highly_variable', 'chrom', 'chromStart', 'chromEnd', 'modality', 'genome'
#     uns: 'neighbors', 'umap', 'cluster'
#     obsm: 'MultiVI', 'X_umap'
#     obsp: 'distances', 'connectivities'

In [6]:
adata.obs['cell_type'] = rna.obs['cell_type'].astype('category')
adata.obs["protocol"] = "SNARE-seq"
adata.obs["dataset"] = "GSE126074-integrated"
adata.var["chrom"] = atac.var["chrom"]
adata.var["chromStart"] = atac.var["chromStart"] 
adata.var["chromEnd"] = atac.var["chromEnd"]
adata.var["genome"] = "mm10"
batch = [name.split('_', 1)[0] for name in rna.obs_names]
adata.obs['batch'] = batch

In [8]:
method_list = ['MultiVI', 'GLUE']
dataset = 'GSE126074'
metrics_list = []
for method in method_list:
    latent = pd.read_csv('../result/'+ dataset + '/'+ method + '.csv', header = None)
    latent.index = adata.obs.index
    adata.obsm[method] = latent
    sc.pp.neighbors(adata, use_rep=method)
    sc.tl.umap(adata)
    sc.tl.leiden(adata, key_added="cluster")
    scib.metrics.cluster_optimal_resolution(adata, cluster_key="cluster", label_key="cell_type")
    ari = scib.metrics.ari(adata, cluster_key="cluster", label_key="cell_type")
    iso_asw = scib.metrics.isolated_labels_asw(adata, label_key="cell_type", batch_key='batch', embed=method,  verbose = False)
    nmi = scib.metrics.nmi(adata, cluster_key="cluster", label_key="cell_type")
    # clisi = scib.metrics.clisi_graph(adata, label_key="cell_type",use_rep=method, type_='embed')
    sht = scib.metrics.silhouette(adata, label_key="cell_type", embed=method, metric='euclidean', scale=True)
    metrics_list.append([ari, iso_asw, nmi, sht, method])  # , clisi

resolution: 0.1, nmi: 0.0032891051703809824
resolution: 0.2, nmi: 0.0032892660483615508
resolution: 0.3, nmi: 0.0037077328412039798
resolution: 0.4, nmi: 0.004586352875722574
resolution: 0.5, nmi: 0.004548146662888701
resolution: 0.6, nmi: 0.005031650871265338
resolution: 0.7, nmi: 0.006735033430123866
resolution: 0.8, nmi: 0.007018197512893286
resolution: 0.9, nmi: 0.008222492010722761
resolution: 1.0, nmi: 0.008915162914103058
resolution: 1.1, nmi: 0.008185398396402595
resolution: 1.2, nmi: 0.009157365471499684
resolution: 1.3, nmi: 0.009901051256699284
resolution: 1.4, nmi: 0.011649258956682756
resolution: 1.5, nmi: 0.012096784407097348
resolution: 1.6, nmi: 0.0112951348633272
resolution: 1.7, nmi: 0.012026129886030775
resolution: 1.8, nmi: 0.011887936100936244
resolution: 1.9, nmi: 0.013514050740847623
resolution: 2.0, nmi: 0.013805421946938012
optimised clustering against cell_type
optimal cluster resolution: 2.0
optimal score: 0.013805421946938012
resolution: 0.1, nmi: 0.17220760

In [15]:
metrics_list

[[-0.00035099532743044343,
  0.4646545301626249,
  0.013805421946938016,
  0.46228627967630437,
  'MultiVI'],
 [0.14058447480197175,
  0.42627553816395086,
  0.33857266763571464,
  0.4935455504014269,
  'GLUE']]

In [19]:
con = mmread('../result/'+ dataset + '/Seurat_connectivities.mtx')
dis = mmread('../result/'+ dataset + '/Seurat_distance.mtx')
adata.uns['neighbors'] = {'connectivities_key': 'connectivities', 'distances_key': 'distances', 
                          'params': {'n_neighbors': 20, 'method': 'umap', 'random_state': 0, 
                                     'metric': 'euclidean'}}
adata.uns['neighbors']['distance'] = csr_matrix(dis)
adata.uns['neighbors']['connectivities'] = csr_matrix(con)
adata.obsp['distance'] = csr_matrix(dis)
adata.obsp['connectivities'] = csr_matrix(con)
sc.tl.umap(adata, n_components=20)
scib.metrics.cluster_optimal_resolution(adata, cluster_key="cluster", label_key="cell_type")
ari = scib.metrics.ari(adata, cluster_key="cluster", label_key="cell_type")
iso_asw = scib.metrics.isolated_labels_asw(adata, label_key="cell_type", batch_key='batch', embed=method,  verbose = False)
nmi = scib.metrics.nmi(adata, cluster_key="cluster", label_key="cell_type")
# clisi = scib.metrics.clisi_graph(adata, label_key="cell_type",use_rep=method, type_='embed')
sht = scib.metrics.silhouette(adata, label_key="cell_type", embed=method, metric='euclidean', scale=True)
metrics_list.append([ari, iso_asw, nmi, sht, 'Seurat'])  # , clisi

resolution: 0.1, nmi: 0.0040183062637716566
resolution: 0.2, nmi: 0.004597223487270575
resolution: 0.3, nmi: 0.006057285597700506
resolution: 0.4, nmi: 0.006798483616871951
resolution: 0.5, nmi: 0.006914551351042559
resolution: 0.6, nmi: 0.0079231476989636
resolution: 0.7, nmi: 0.008977425218307444
resolution: 0.8, nmi: 0.009180543386989471
resolution: 0.9, nmi: 0.009180872094939129
resolution: 1.0, nmi: 0.009712737994340822
resolution: 1.1, nmi: 0.009864218536136863
resolution: 1.2, nmi: 0.010054471600565083
resolution: 1.3, nmi: 0.010507220972004823
resolution: 1.4, nmi: 0.010131881133354365
resolution: 1.5, nmi: 0.010652294660758213
resolution: 1.6, nmi: 0.011598754127211884
resolution: 1.7, nmi: 0.011475227749335248
resolution: 1.8, nmi: 0.011516359818942931
resolution: 1.9, nmi: 0.011909649636472343
resolution: 2.0, nmi: 0.011965183395344229
optimised clustering against cell_type
optimal cluster resolution: 2.0
optimal score: 0.011965183395344229


In [21]:
# df = pd.DataFrame(metrics_list, columns = ['','method'])
result = pd.DataFrame()
df = pd.DataFrame(metrics_list, columns = ['ari', 'iso_asw','nmi','sht','method'])
result = pd.concat([result, df], ignore_index=True)

result.to_csv("../result/GSE126074/metrics_result.csv", index=False)